## day 3


In [14]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import tensorflow as tf
import keras


prices=pd.read_csv('/fb-stock-prices.csv')
prices.head()

prices.dtypes

plt.figure(figsize=(20,10))
plt.plot(prices['Price'])
plt.show()

prices.count()

# DATA PREPROCESSING


from sklearn.preprocessing import StandardScaler,minmax_scale
scalar = StandardScaler()
# Changed 'prices' to 'Price' to match the actual column name in the DataFrame
scaled_prices = scalar.fit_transform(prices[["Price"]].values)

print(scaled_prices)

total_size=len(scaled_prices)
test_size=50
train_size=total_size-test_size

print(total_size)
print(test_size)
print(train_size)

training_prices=scaled_prices[:train_size]
test_prices = scaled_prices[train_size:]

print(training_prices.shape)

def create_rnn_dataset(data, look_back=1):
    data_x=[]
    data_y=[]
    for i in range(len(data)-look_back-1):
        b=data[i:(i+look_back),0]
        data_x.append(b)
        data_y.append(data[i+look_back,0])
    return numpy.array(data_x),numpy.array(data_y)

lookback=25
train_req_x,train_req_y=create_rnn_dataset(training_prices,lookback)
train_req_x.shape

train_req_y.shape

train_req_x=numpy.reshape(train_req_x,(train_req_x.shape[0],1,train_req_x.shape[1]))

print(train_req_x.shape)


from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
import tensorflow as tf

price_model = Sequential()
price_model.add(SimpleRNN(32,input_shape=(1,lookback)))
price_model.add(Dense(1))
price_model.compile(loss='mean_squared_error',optimizer='adam')


price_model.fit(train_req_x,train_req_y,epochs=5,batch_size=5,verbose=1)

test_req_x,test_req_y=create_rnn_dataset(test_prices,lookback)
test_req_x = numpy.reshape(test_req_x,(test_req_x.shape[0],1,test_req_x.shape[1]))

predictions
prev_prices = numpy.array([324,356,456,321,324,
                        345,330,331,335,326,
                        345,330,333,354,356,
                        349,330,331,335,326,
                        356,331,335,349,333])
scaled_prices = scaler.transform(prev_prices.reshape(-1,1))
model_input = numpy.reshape(scaled_prices,(scaled_prices.shape[1],1,scaled_prices.shape[0]))
pred = price_model.predict(model_input)
stock_pred = scaler.inverse_transform(pred)
print(stock_pred)

ModuleNotFoundError: No module named 'matplotlib'

## Day 3 LSTM exercise

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Load dataset
file_path = '/mnt/data/Spam-Classification.csv'
data = pd.read_csv(file_path)

# Preprocess labels
label_encoder = LabelEncoder()
data['CLASS'] = label_encoder.fit_transform(data['CLASS'])  # 0 for ham, 1 for spam

# Tokenize SMS content
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(data['SMS'])
word_index = tokenizer.word_index

# Convert SMS to sequences
sequences = tokenizer.texts_to_sequences(data['SMS'])

# Pad sequences
max_length = 100  # Maximum length of SMS messages
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Prepare train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, data['CLASS'], test_size=0.3, random_state=42)

# Define LSTM model
vocab_size = len(word_index) + 1  # Total number of unique words
embedding_dim = 64

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
batch_size = 32
epochs = 5
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=epochs)

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Make predictions
predictions = model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int)

# Display prediction results
print("Sample Predictions:")
for i in range(10):  # Display 10 sample predictions
    print(f"Message: {data['SMS'].iloc[i]}\nActual: {y_test.iloc[i]}\nPredicted: {predicted_classes[i][0]}\n")


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/Spam-Classification.csv'